# 建立sample:name字典

In [1]:
with open('sample_name.txt','r') as f:
    name = f.read().split("\n")
    name_dict = {}
    for i in name:
        key, value = i.split(":")
        name_dict[key] = value
    number_of_samples = len(name_dict)

# 載入訓練好的模型

In [2]:
from keras.models import load_model
classifier = load_model('SaveModel/face.hd5')

Using TensorFlow backend.


# 人臉辨識

In [3]:
import numpy as np          
import dlib                
import cv2             

def facerecognition(film): 
    cap = cv2.VideoCapture(film)                               
    detector = dlib.get_frontal_face_detector()             

    while(cap.isOpened()):     
        ret, frame = cap.read()  
        face_rects, scores, idx = detector.run(frame, 0)    
        for i, d in enumerate(face_rects):                  
            x1 = d.left()
            y1 = d.top()
            x2 = d.right()
            y2 = d.bottom()
            cropped = frame[int(y1):int(y2),int(x1):int(x2)] 
            image=cv2.resize(cropped,(64, 64),interpolation=cv2.INTER_CUBIC)
            image = np.expand_dims(image, axis = 0)        
            label = str(classifier.predict_classes(image)[0])
            label = name_dict['sample'+label]               
            cv2.rectangle(frame, (x1, y1), (x2, y2), (0, 255, 0), 4, cv2.LINE_AA) 
            cv2.putText(frame, label, (x1, y1), cv2.FONT_HERSHEY_DUPLEX, 0.7, (255, 255, 255), 1, cv2.LINE_AA)  
        cv2.imshow("face recognition", frame)                  
        if cv2.waitKey(1) & 0xFF == ord('q'):              
            break

    cap.release()                                           
    cv2.destroyAllWindows()                                 

In [7]:
facerecognition('lo.mp4')  